# Class level relationships

## Generalization

We need a target project rather than a single file.

In [ ]:
# Clone a target project. This time, we try Mojang/brigadier, a MineCraft program.
!git clone git@github.com:Mojang/brigadier.git

In [ ]:
# First, we need to read many java files.
# from os.path import isfile, join
import os

path = "./brigadier/src/main/java/**/*.java"

from glob import glob


for filename in glob(path, recursive=True):
    print(filename)

In [3]:
javafiles = list(glob(path, recursive=True))
len(javafiles)

47

In [4]:
## Parser function
import javalang

def getAST(path: str):
    with open(path, "r") as f:
        code = f.read()
    return javalang.parse.parse(code)

## Get AST one-by-one

In [24]:
# print(javafiles[0])
myast = getAST(javafiles[14])
# myast

## Detect `extends` and `implements`

In [37]:
gen_real_dict = {}

for javafile in javafiles:
    myast = getAST(javafile)
    for path, node in javalang.ast.walk_tree(myast):
        if isinstance(node, javalang.tree.ClassDeclaration):
            gen_real_dict[node.name] = { "extends": node.extends.name if node.extends is not None else None,
                                         "implements": list([atype.name for atype in node.implements]) if node.implements is not None else None,
                                         "type": "class"}
#             print("=====")
#             print("Class:", node.name)
#             print("Extends:", node.extends)
#             print("Implements:", node.implements)
        elif isinstance(node, javalang.tree.InterfaceDeclaration):
            gen_real_dict[node.name] = { "extends": node.extends.name if node.extends is not None else None,
                                         "type": "interface"}
#             print("=====")
#             print("Interface:", node.name)
#             print("Extends:", node.extends)
        elif isinstance(node, javalang.tree.AnnotationDeclaration):
            gen_real_dict[node.name] = {}
#             print("=====")
#             print("Annotation:", node.name)
        elif isinstance(node, javalang.tree.EnumDeclaration):
            gen_real_dict[node.name] = { "implements": list([atype.name for atype in node.implements]) if node.implements is not None else None,
                                         "type": "enum"}
#             print("=====")
#             print("Enum:", node.name)
#             print("Implements:", node.implements)    

In [45]:
# gen_real_dict
for key, value in gen_real_dict.items():
    if value["type"] == "class":
        if value["extends"] is not None:
            gen_real_dict[key]["dangling"] = False
            if value["extends"] in gen_real_dict:
                gen_real_dict[value["extends"]]["dangling"] = False
        if value["implements"] is not None:
            for aiterface in value["implements"]:
                gen_real_dict[key]["dangling"] = False
                if aiterface in gen_real_dict:
                    gen_real_dict[aiterface]["dangling"] = False
    elif value["type"] == "interface":
        if value["extends"] is not None:
            gen_real_dict[key]["dangling"] = False
            if value["extends"] in gen_real_dict:
                gen_real_dict[value["extends"]]["dangling"] = False
#     if value["implements"] is not None and len(value["implements"]) > 1:
#         print(f"{key}:", value["implements"])

for key, value in gen_real_dict.items():
    if "dangling" not in value:
        print(f"{key} is dangling")

SingleRedirectModifier is dangling
ResultConsumer is dangling
Command is dangling
ParseResults is dangling
RedirectModifier is dangling
AmbiguityConsumer is dangling
CommandDispatcher is dangling
ParsedArgument is dangling
SuggestionContext is dangling
CommandContextBuilder is dangling
ContextChain is dangling
Stage is dangling
StringRange is dangling
ParsedCommandNode is dangling
CommandContext is dangling
SuggestionProvider is dangling
Suggestions is dangling
SuggestionsBuilder is dangling
Function is dangling
StringType is dangling


## Store generalization relationships in a data structure

In [36]:
for key, value in gen_real_dict.items():
    if value["extends"] is not None:
        parent = value["extends"]
        print(f"{key} -> {parent}")

ArgumentCommandNode -> CommandNode
LiteralCommandNode -> CommandNode
RootCommandNode -> CommandNode
IntegerSuggestion -> Suggestion
CommandSyntaxException -> Exception
RequiredArgumentBuilder -> ArgumentBuilder
LiteralArgumentBuilder -> ArgumentBuilder


# Association

## Composition

## Aggregation